In [ ]:
import axelrod as axl
import numpy as np
import pandas as pd

R, S, T, P = 3, 0, 5, 1

In [ ]:
def simulate_match(strategy_class, opponent_class, turns=1000):
    players = [strategy_class(), opponent_class()]
    match = axl.Match(players, turns=turns)
    match.play()
    result = match.result
    moves_strategy = [str(turn[0]) for turn in result]
    moves_opponent = [str(turn[1]) for turn in result]
    return moves_strategy, moves_opponent


def evaluate_niceness(moves):
    return 1 if moves[0] == 'C' else 0

def evaluate_retaliation(moves_strategy, moves_opponent):
    retaliations = 0
    opportunities = 0
    for i in range(1, len(moves_strategy)):
        if moves_opponent[i-1] == 'D' and moves_strategy[i-1] == 'C':
            opportunities += 1
            if moves_strategy[i] == 'D':
                retaliations += 1
    return retaliations / opportunities if opportunities > 0 else None

def evaluate_forgiveness(moves_strategy, moves_opponent):
    forgiveness_count = 0
    opportunities = 0
    for i in range(2, len(moves_strategy)):
        if moves_opponent[i-2] == 'D' and moves_strategy[i-1] == 'D':
            if moves_opponent[i-1] == 'C':
                opportunities += 1
                if moves_strategy[i] == 'C':
                    forgiveness_count += 1
    return forgiveness_count / opportunities if opportunities > 0 else None

def evaluate_overall_cooperation(moves):
    moves_str = [str(move) for move in moves]
    return moves_str.count('C') / len(moves_str)

def evaluate_average_payoff(moves_strategy, moves_opponent):
    score = 0
    for a, b in zip(moves_strategy, moves_opponent):
        if a == 'C' and b == 'C':
            score += R
        elif a == 'C' and b == 'D':
            score += S
        elif a == 'D' and b == 'C':
            score += T
        else:
            score += P
    return score / len(moves_strategy)

In [ ]:
def get_deterministic_strategies():
    all_strategies = axl.all_strategies
    return [strategy for strategy in all_strategies if not strategy.classifier['stochastic']]

deterministic_strategies = sorted(get_deterministic_strategies(), key=lambda s: s.__name__)
pool_strategies = deterministic_strategies

strategy_mapping = {}
for strategy in pool_strategies:
      strategy_mapping[strategy.__name__] = strategy

print(strategy_mapping.keys())

strategy_names = list(strategy_mapping.keys())
print("Total strategies in pool:", (strategy_names))

dict_keys(['AON2', 'APavlov2006', 'APavlov2011', 'Adaptive', 'AdaptiveTitForTat', 'Aggravater', 'Alexei', 'Alternator', 'AlternatorHunter', 'AntiCycler', 'AntiTitForTat', 'Appeaser', 'BackStabber', 'Bully', 'Capri', 'CollectiveStrategy', 'ContriteTitForTat', 'Cooperator', 'CooperatorHunter', 'CycleHunter', 'CyclerCCCCCD', 'CyclerCCCD', 'CyclerCCCDCD', 'CyclerCCD', 'CyclerDC', 'CyclerDDC', 'DBS', 'Darwin', 'Defector', 'DefectorHunter', 'DelayedAON1', 'Detective', 'DoubleCrosser', 'DoubleResurrection', 'Doubler', 'EasyGo', 'EugineNier', 'EventualCycleHunter', 'EvolvedANN', 'EvolvedANN5', 'EvolvedANNNoise05', 'EvolvedFSM16', 'EvolvedFSM16Noise05', 'EvolvedFSM4', 'EvolvedFSM6', 'EvolvedLookerUp1_1_1', 'EvolvedLookerUp2_2_2', 'FirstByDavis', 'FirstByDowning', 'FirstByNydegger', 'FirstByShubik', 'FirstBySteinAndRapoport', 'FirstByTidemanAndChieruzzi', 'FoolMeOnce', 'ForgetfulGrudger', 'Forgiver', 'ForgivingTitForTat', 'Fortress3', 'Fortress4', 'GeneralSoftGrudger', 'GoByMajority', 'GoByMajor

In [ ]:
num_repeats = 7

metrics_accum = { strat: {"niceness": [], "retaliation": [], "forgiveness": [], "overall_coop": [], "avg_payoff": []}
                  for strat in strategy_names }

n = len(strategy_names)
counter = 0
for i in range(n):
    for j in range(i, n):
        strat_s = strategy_names[i]
        strat_t = strategy_names[j]

        niceness_vals_s = []
        retaliation_vals_s = []
        forgiveness_vals_s = []
        overall_coop_vals_s = []
        avg_payoff_vals_s = []

        niceness_vals_t = []
        retaliation_vals_t = []
        forgiveness_vals_t = []
        overall_coop_vals_t = []
        avg_payoff_vals_t = []

        for r in range(num_repeats):
            counter += 1
            print(f"Match {counter}: {strat_s} vs. {strat_t}")
            moves_s, moves_t = simulate_match(strategy_mapping[strat_s], strategy_mapping[strat_t], turns=1000)

            n_val = 1 if moves_s[0]=='C' else 0
            r_val = evaluate_retaliation(moves_s, moves_t)
            f_val = evaluate_forgiveness(moves_s, moves_t)
            oc_val = moves_s.count('C') / len(moves_s)
            ap_val = evaluate_average_payoff(moves_s, moves_t)

            niceness_vals_s.append(n_val)
            if r_val is not None:
                retaliation_vals_s.append(r_val)
            if f_val is not None:
                forgiveness_vals_s.append(f_val)
            overall_coop_vals_s.append(oc_val)
            avg_payoff_vals_s.append(ap_val)

            if strat_s != strat_t:
                n_val_t = 1 if moves_t[0]=='C' else 0
                r_val_t = evaluate_retaliation(moves_t, moves_s)
                f_val_t = evaluate_forgiveness(moves_t, moves_s)
                oc_val_t = moves_t.count('C') / len(moves_t)
                ap_val_t = evaluate_average_payoff(moves_t, moves_s)

                niceness_vals_t.append(n_val_t)
                if r_val_t is not None:
                    retaliation_vals_t.append(r_val_t)
                if f_val_t is not None:
                    forgiveness_vals_t.append(f_val_t)
                overall_coop_vals_t.append(oc_val_t)
                avg_payoff_vals_t.append(ap_val_t)

        metrics_accum[strat_s]["niceness"].append(np.mean(niceness_vals_s))
        if retaliation_vals_s:
            metrics_accum[strat_s]["retaliation"].append(np.mean(retaliation_vals_s))
        if forgiveness_vals_s:
            metrics_accum[strat_s]["forgiveness"].append(np.mean(forgiveness_vals_s))
        metrics_accum[strat_s]["overall_coop"].append(np.mean(overall_coop_vals_s))
        metrics_accum[strat_s]["avg_payoff"].append(np.mean(avg_payoff_vals_s))

        if strat_s != strat_t:
            metrics_accum[strat_t]["niceness"].append(np.mean(niceness_vals_t))
            if retaliation_vals_t:
                metrics_accum[strat_t]["retaliation"].append(np.mean(retaliation_vals_t))
            if forgiveness_vals_t:
                metrics_accum[strat_t]["forgiveness"].append(np.mean(forgiveness_vals_t))
            metrics_accum[strat_t]["overall_coop"].append(np.mean(overall_coop_vals_t))
            metrics_accum[strat_t]["avg_payoff"].append(np.mean(avg_payoff_vals_t))


metrics_list = []
for strat in strategy_names:
    acc = metrics_accum[strat]
    avg_niceness = np.mean(acc["niceness"]) if acc["niceness"] else None
    avg_retaliation = np.mean(acc["retaliation"]) if acc["retaliation"] else None
    avg_forgiveness = np.mean(acc["forgiveness"]) if acc["forgiveness"] else None
    avg_overall_coop = np.mean(acc["overall_coop"]) if acc["overall_coop"] else None
    avg_avg_payoff = np.mean(acc["avg_payoff"]) if acc["avg_payoff"] else None
    metrics_list.append({
         "Strategy": strat,
         "Niceness": avg_niceness,
         "Retaliation": avg_retaliation,
         "Forgiveness": avg_forgiveness,
         "OverallCoop": avg_overall_coop,
         "AvgPayoff": avg_avg_payoff
    })

tournament_metrics = pd.DataFrame(metrics_list)
print(tournament_metrics)

Streaming output truncated to the last 5000 lines.
Match 76424: Second by Grofman vs. UsuallyCooperates
Match 76425: Second by Grofman vs. UsuallyCooperates
Match 76426: Second by Grofman vs. UsuallyCooperates
Match 76427: Second by Grofman vs. UsuallyDefects
Match 76428: Second by Grofman vs. UsuallyDefects
Match 76429: Second by Grofman vs. UsuallyDefects
Match 76430: Second by Grofman vs. UsuallyDefects
Match 76431: Second by Grofman vs. UsuallyDefects
Match 76432: Second by Grofman vs. UsuallyDefects
Match 76433: Second by Grofman vs. UsuallyDefects
Match 76434: Second by Grofman vs. VeryBad
Match 76435: Second by Grofman vs. VeryBad
Match 76436: Second by Grofman vs. VeryBad
Match 76437: Second by Grofman vs. VeryBad
Match 76438: Second by Grofman vs. VeryBad
Match 76439: Second by Grofman vs. VeryBad
Match 76440: Second by Grofman vs. VeryBad
Match 76441: Second by Grofman vs. Win-Shift Lose-Stay
Match 76442: Second by Grofman vs. Win-Shift Lose-Stay
Match 76443: Second by Grofma

In [ ]:
tournament_metrics

,Strategy,Niceness,Retaliation,Forgiveness,OverallCoop,AvgPayoff
0,AON2,1.0,0.904852,0.000000,0.713638,2.780520
1,Adaptive Pavlov 2006,1.0,0.939072,0.676181,0.760546,2.685270
2,Adaptive Pavlov 2011,1.0,0.881374,0.703772,0.778711,2.683237
3,Adaptive,1.0,0.082421,0.067308,0.496782,2.239187
4,Adaptive Tit For Tat,1.0,0.934615,1.000000,0.834040,2.725786
...,...,...,...,...,...,...
147,Win-Shift Lose-Stay,0.0,0.000000,1.000000,0.562132,1.867079
148,Win-Stay Lose-Shift,1.0,1.000000,0.000000,0.793692,2.581115
149,Winner12,1.0,0.744292,0.000000,0.725276,2.863598
150,Winner21,0.0,1.000000,0.000000,0.363242,2.028095


In [ ]:
tournament_metrics.to_csv('..//simulated_data/precalculated_metrics/all_strategy_metrics.csv.csv', index=False)
